In [19]:
import json
import requests
import time

with open('./api_parsing/token.json') as f:
    data = json.load(f)

IP = data['ip']
TOKEN = data['access_token']

def get_user_friends(username, params):
    response = requests.get(f'https://api.jikan.moe/v4/users/{username}/friends', params=params)
    if response.status_code == 200:
        time.sleep(0.5)
        return response.json()
    else:
        print('REQUEST ERROR')
        time.sleep(1)
        return None
    
def get_user_list(username, params):
    global TOKEN

    headers = {'Authorization': f'Bearer {TOKEN}'}

    response = requests.get(f'https://api.myanimelist.net/v2/users/{username}/animelist', params=params, headers=headers)
    if response.status_code == 200:
        time.sleep(0.5)
        return response.json()
    else:
        print('REQUEST ERROR')
        time.sleep(1)
        return None

In [3]:
from pymongo import MongoClient, errors

client = MongoClient("mongodb://localhost", 27017)

db = client['AnimeRecs']

col = db['UserNames']

try:
    col.create_index([('name', 1)], unique=True)
except errors.DuplicateKeyError as e:
    print(f"Error: {e}")

In [4]:
#try insertion of name
def insertName(name):
    try:
        col.insert_one({'name': name})
    except errors.DuplicateKeyError:
        print(f'{name} already exists')

In [24]:
#main script for data collection

#max value for names to be collected
max_value = 10000

#starting names (hardcoded)
starting_names = ['Stark700', 'Kanashimi', 'Veronin', 'Sorrowful', 'Decoys']

for current in starting_names:

    #get friends of the first name
    friends = get_user_friends(current, {})
    more_pages = False
    page = 1
    if friends['pagination']['has_next_page']:
        more_pages = True
        #find additional pages of friends
        while more_pages:
            page += 1 
            new_friends = get_user_friends(current, {'page': page})
            more_pages = new_friends['pagination']['has_next_page']
            friendlist = friendlist + [friend['user']['username'] for friend in new_friends['data']]
    else:
        friendlist = [friend['user']['username'] for friend in friends['data']]

    #check for dupes
    friendlist = [friend not in starting_names for friend in friendlist]

    #put each of those friends into a list
    if (starting_names < max_value):
        starting_names = starting_names + friendlist

    #put into db
    insertName(current)

[]